In [1]:
#Esto me sirve para el correcto funcionamiento de las funciones importadas en este notebook
%load_ext autoreload
%autoreload 2

# Analizando los datos

Como primer paso del análisis miramos las correlaciones del resto de variables con respecto a la variable objetivo la cual es **'mvp_share'**. Esto nos servirá para tener una idea inicial sobre a cuales features darles importancia o que esperar sobre nuestro modelo.

In [ ]:
corr_matrix = df_data_jugadores_train.corr()
print(corr_matrix["mvp_share"].sort_values(ascending=False).head(15))

Podemos observar que entre las características que presentan una mayor correlación con nuestra variable objetivo se encuentran:

*   **'vorp' (Value over Replacement Player)**: Es una estadística avanzada que busca cuantificar el valor total que un jugador aporta a su equipo en comparación con un jugador genérico de reemplazo que estaría disponible para jugar en su lugar.

*   **'bpm' (Box Plus/Minus)**: Calcula el más/menos de un jugador respecto al rendimiento del equipo por cada 100 posesiones.

*   **'per' (Player Efficiency Rating)**: Una fórmula que calcula la **eficencia de un jugador en base a lo que produce por minuto**. El PER toma en cuenta los aciertos en tiros de campo, tiros libres, triples, asistencias, rebotes, tapones y robos, y los errores como tiros fallados, pérdidas de balón y faltas personales. La fórmula agrega estadísticas positivas y resta las negativas a través de un sistema de valores de puntos estadísticos. Este sistema fue **desarrollado por [John Hollinger](https://en.wikipedia.org/wiki/John_Hollinger)**.

*   **'ws' (Win Shares)**: Indice que reparte el éxito de un equipo entre cada uno de los individuos de ese equipo, estimando el número de victorias que genera un jugador en base a estadísticas individuales, de equipo y de toda la Liga.

*   **'ws_48' (Win Shares per 48 min)**:
Es igual a la estadística definida anteriormente, pero esta se basa en 48 minutos porque esa es la duración exacta de un partido de la NBA. Por lo tanto, esta estadística puede medir cuánto contribuye un jugador a un esfuerzo ganador en función de su desempeño por minuto en cada partido.

*   **'obpm' (Offensive Box Plus/Minus)**: El Box Plus/Minus ofensivo (OBPM) es una estadística en baloncesto que mide la contribución de un jugador en términos de puntos por encima del promedio de la liga por cada 100 posesiones jugadas, centrándose exclusivamente en su rendimiento ofensivo. Muestra cuánto un jugador mejora la producción ofensiva de su equipo cuando está en la cancha.

La alta correlación de estas características con la variable objetivo no es coincidencia, ya que concuerda con lo que mencionamos anteriormente sobre el significado de **ser el jugador más valioso**. Se refiere a un jugador que tiene un impacto positivo significativo en su equipo en comparación con sus compañeros.

Ahora con esto en mente, procedemos a ver de forma gráfica estas correlaciones para darnos una idea más clara de lo que está ocurriendo:

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(14, 8))

columnas = ['vorp', 'bpm', 'per', 'ws', 'ws_48', 'obpm']
titulos = ['Value over Replacement Player', 'Box Plus/Minus', 'Player Efficiency Rating', 'Win Shares', 'Win Shares per 48 min', 'Offensive Box Plus/Minus']

for i, col in enumerate(columnas):
    sns.scatterplot(x=col, y='mvp_share', data=df_data_jugadores_train, ax=axes[i // 3, i % 3])
    axes[i // 3, i % 3].set_title(f'{titulos[i]} vs MVP shares')

plt.tight_layout()

Podemos notar que en los cinco atributos, la relación con **'mvp_share'** es positiva y ascendente. Este gráfico nos brinda una herramienta valiosa para identificar los **"destacados"** entre los jugadores, aquellos que recibieron una considerable cantidad de votos, lo que sugiere que tuvieron temporadas excepcionales.

Ahora analizaremos como se distribuyen estos datos para detectar la existencia de grupos entre los jugadores con respecto a estos atributos:

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(14, 8))

columnas = ['vorp', 'bpm', 'per', 'ws', 'ws_48', 'obpm']
titulos = ['Value over Replacement Player', 'Box Plus/Minus', 'Player Efficiency Rating', 'Win Shares', 'Win Shares per 48 min', 'Offensive Box Plus/Minus']

for i, col in enumerate(columnas):
    axes[i // 3, i % 3].hist(df_data_jugadores_train[col], bins=20, color='skyblue', edgecolor='black')
    axes[i // 3, i % 3].set_title(f'{titulos[i]}')

plt.tight_layout()

Todo sugiere que tenemos una distribución uniforme en las variables de interés. Esto indica que, hasta el momento, nuestros datos no presentan ningún problema aparente.

Luego de un exhaustivo análisis de datos, optamos por seleccionar features **basándonos en la correlación** que presentan con nuestro objetivo. Inicialmente, nos enfocaremos en aquellas con una correlación **superior a 0.25**. Aunque solo se presentan algunas, estas variables parecen ser predictores valiosos, ya que cada una resalta un aspecto específico del jugador. En consecuencia, hemos decidido emplear las **14 features** que cumplen con la correlación que deseamos.

In [ ]:
correlated_features = corr_matrix["mvp_share"].sort_values(ascending=False).index.tolist()[1:15]